# This notebook contains all processes to import/cleanse/validate and prepare data for processing for the remainder of the project.

## Bring in raw dataset

In [1]:
import pandas as pd

df_movies_raw = pd.read_csv('./data/raw/movies.csv')

df_movies_raw.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


### check shape for validation throughout process 
(7668, 25)

In [2]:
# check the shape of the dataframe
df_movies_raw.shape   #7668,15, which matches the shape of the original dataset

(7668, 15)

## Preliminary investitation 

### Dataset conventions
* Are gross and budget amounts in US dollars or foreign currency for foreign films?  
     * Confirmed that all amounts shown are in US dollars by comparing documented gross within the individdual IMDB pages for films, which shows US $$, and confirmed that total matches the dataset
* compare the budget/gross to historic online data to determine if the inflation adjustments were already done
    * "Jaws 3D" shows $88M gross in both dataset and on wikipedia https://en.wikipedia.org/wiki/Jaws_3-D
    * "Things are Tough All Over" shows $21M gross in both dataset and wikipedia https://en.wikipedia.org/wiki/Things_Are_Tough_All_Over
    * This seems to indicate that the dataset has not already been adjusted for inflation, meaning we need to as part of our analysis

* Determine if data was inconsistently entered in a way that might separate attribution; e.g. Star Robert Redford (8), Star Robbert Redford (1); 
    * do this for fields ['star', 'writer', 'director', 'country', 'genre', 'company']
 

### Data cleanliness


In [4]:
# Duplicate movie names
# give me a list of all movies where name appears > 1 times, include all fields for those items, order by name
df_duplicates = df_movies_raw[df_movies_raw.duplicated(subset=['name'], keep=False)].sort_values(by=['name'])
#write df_duplicates to csv file
df_duplicates.to_csv('./data/raw/duplicate_movies.csv', index=False)





* Determine if there are duplicated movie titles
    * any duplication of titles is due to remakes/sequels

## Data Shaping

### Create separate field for release year in order to join to the inflation multiplier data
* clean the release date to be a standard date by separating into
    * release_date
    * country

In [5]:

# split the 'released' column into 'release_date' and 'country'
df_movies_raw[['release_date', 'country']] = df_movies_raw['released'].str.extract(r'^(.*?)(?:\s*\((.*?)\))?$')

# transform release_date to datetime
df_movies_raw['release_date'] = pd.to_datetime(df_movies_raw['release_date'], errors='coerce')
df_movies_raw.T




,0,1,2,3,4,5,6,7,8,9,...,7658,7659,7660,7661,7662,7663,7664,7665,7666,7667
name,The Shining,The Blue Lagoon,Star Wars: Episode V - The Empire Strikes Back,Airplane!,Caddyshack,Friday the 13th,The Blues Brothers,Raging Bull,Superman II,The Long Riders,...,Black Wall Street Burning,I Am Fear,Aloha Surf Hotel,Love by Drowning,The Robinsons,More to Life,Dream Round,Saving Mbango,It's Just Us,Tee em el
rating,R,R,PG,PG,R,R,R,R,PG,R,...,R,Not Rated,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN
genre,Drama,Adventure,Action,Comedy,Comedy,Horror,Action,Biography,Action,Biography,...,Drama,Horror,Comedy,Drama,Action,Drama,Comedy,Drama,Drama,Horror
year,1980,1980,1980,1980,1980,1980,1980,1980,1980,1980,...,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020
released,"June 13, 1980 (United States)","July 2, 1980 (United States)","June 20, 1980 (United States)","July 2, 1980 (United States)","July 25, 1980 (United States)","May 9, 1980 (United States)","June 20, 1980 (United States)","December 19, 1980 (United States)","June 19, 1981 (United States)","May 16, 1980 (United States)",...,"February 7, 2020 (United States)","March 3, 2020 (United States)","November 5, 2020 (United States)","November 6, 2020 (United States)","November 10, 2020 (United States)","October 23, 2020 (United States)","February 7, 2020 (United States)","April 27, 2020 (Cameroon)","October 1, 2020 (United States)","August 19, 2020 (United States)"
score,8.4,5.8,8.7,7.7,7.3,6.4,7.9,8.2,6.8,7.0,...,6.6,3.4,7.1,NaN,NaN,3.1,4.7,5.7,NaN,5.7
votes,927000.0,65000.0,1200000.0,221000.0,108000.0,123000.0,188000.0,330000.0,101000.0,10000.0,...,35.0,447.0,14.0,NaN,NaN,18.0,36.0,29.0,NaN,7.0
director,Stanley Kubrick,Randal Kleiser,Irvin Kershner,Jim Abrahams,Harold Ramis,Sean S. Cunningham,John Landis,Martin Scorsese,Richard Lester,Walter Hill,...,Marcus Brown,Kevin Shulman,Stefan C. Schaefer,Justin Kreinbrink,Directors,Joseph Ebanks,Dusty Dukatz,Nkanya Nkwai,James Randall,Pereko Mosia
writer,Stephen King,Henry De Vere Stacpoole,Leigh Brackett,Jim Abrahams,Brian Doyle-Murray,Victor Miller,Dan Aykroyd,Jake LaMotta,Jerry Siegel,Bill Bryden,...,Dekoven Riggins,Kevin Shulman,Stefan C. Schaefer,C.E. Poverman,Aleks Alifirenko Jr.,Joseph Ebanks,Lisa Huston,Lynno Lovert,James Randall,Pereko Mosia
star,Jack Nicholson,Brooke Shields,Mark Hamill,Robert Hays,Chevy Chase,Betsy Palmer,John Belushi,Robert De Niro,Gene Hackman,David Carradine,...,Dan Belcher,Kristina Klebe,Augie Tulba,Nicky Whelan,Billy Hartmann,Shannon Bond,Michael Saquella,Onyama Laura,Christina Roz,Siyabonga Mabaso


### Add field for [decade] in order to reduce the dimensionality of year

In [6]:
# add a field to df_movies_raw called [decade] that is the decade of the release_date, e.g. 1990s, 2000s, etc.
df_movies_raw['decade'] = (df_movies_raw['release_date'].dt.year // 10 * 10).astype('Int64').astype(str) + 's'
df_movies_raw.T

,0,1,2,3,4,5,6,7,8,9,...,7658,7659,7660,7661,7662,7663,7664,7665,7666,7667
name,The Shining,The Blue Lagoon,Star Wars: Episode V - The Empire Strikes Back,Airplane!,Caddyshack,Friday the 13th,The Blues Brothers,Raging Bull,Superman II,The Long Riders,...,Black Wall Street Burning,I Am Fear,Aloha Surf Hotel,Love by Drowning,The Robinsons,More to Life,Dream Round,Saving Mbango,It's Just Us,Tee em el
rating,R,R,PG,PG,R,R,R,R,PG,R,...,R,Not Rated,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN
genre,Drama,Adventure,Action,Comedy,Comedy,Horror,Action,Biography,Action,Biography,...,Drama,Horror,Comedy,Drama,Action,Drama,Comedy,Drama,Drama,Horror
year,1980,1980,1980,1980,1980,1980,1980,1980,1980,1980,...,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020
released,"June 13, 1980 (United States)","July 2, 1980 (United States)","June 20, 1980 (United States)","July 2, 1980 (United States)","July 25, 1980 (United States)","May 9, 1980 (United States)","June 20, 1980 (United States)","December 19, 1980 (United States)","June 19, 1981 (United States)","May 16, 1980 (United States)",...,"February 7, 2020 (United States)","March 3, 2020 (United States)","November 5, 2020 (United States)","November 6, 2020 (United States)","November 10, 2020 (United States)","October 23, 2020 (United States)","February 7, 2020 (United States)","April 27, 2020 (Cameroon)","October 1, 2020 (United States)","August 19, 2020 (United States)"
score,8.4,5.8,8.7,7.7,7.3,6.4,7.9,8.2,6.8,7.0,...,6.6,3.4,7.1,NaN,NaN,3.1,4.7,5.7,NaN,5.7
votes,927000.0,65000.0,1200000.0,221000.0,108000.0,123000.0,188000.0,330000.0,101000.0,10000.0,...,35.0,447.0,14.0,NaN,NaN,18.0,36.0,29.0,NaN,7.0
director,Stanley Kubrick,Randal Kleiser,Irvin Kershner,Jim Abrahams,Harold Ramis,Sean S. Cunningham,John Landis,Martin Scorsese,Richard Lester,Walter Hill,...,Marcus Brown,Kevin Shulman,Stefan C. Schaefer,Justin Kreinbrink,Directors,Joseph Ebanks,Dusty Dukatz,Nkanya Nkwai,James Randall,Pereko Mosia
writer,Stephen King,Henry De Vere Stacpoole,Leigh Brackett,Jim Abrahams,Brian Doyle-Murray,Victor Miller,Dan Aykroyd,Jake LaMotta,Jerry Siegel,Bill Bryden,...,Dekoven Riggins,Kevin Shulman,Stefan C. Schaefer,C.E. Poverman,Aleks Alifirenko Jr.,Joseph Ebanks,Lisa Huston,Lynno Lovert,James Randall,Pereko Mosia
star,Jack Nicholson,Brooke Shields,Mark Hamill,Robert Hays,Chevy Chase,Betsy Palmer,John Belushi,Robert De Niro,Gene Hackman,David Carradine,...,Dan Belcher,Kristina Klebe,Augie Tulba,Nicky Whelan,Billy Hartmann,Shannon Bond,Michael Saquella,Onyama Laura,Christina Roz,Siyabonga Mabaso


### Adjust gross budget and revenue dollars for inflation
* Merged data with ./data/raw/adjusted_dollars.csv to create new fields [adjusted_gross] and [adjusted_budget]

In [7]:
# get adjusted_dollars.csv into a dataframe
df_adjusted_dollars = pd.read_csv('./data/raw/adjusted_dollars.csv')
#df_adjusted_dollars.shape


# merge df_movies_raw with df_adjusted_dollars on year and add InflationMultiplier field to df_movies_raw
df_movies = pd.merge(df_movies_raw, df_adjusted_dollars[['Year', 'InflationMultiplier']], left_on=df_movies_raw['year'], right_on='Year', how='left')
# drop the redundant Year field
df_movies = df_movies.drop(columns=['Year'])

# add a new column for adjusted gross
df_movies['adjusted_gross'] = df_movies['gross'] * df_movies['InflationMultiplier']
df_movies['adjusted_budget'] = df_movies['budget'] * df_movies['InflationMultiplier']


In [8]:
# ensure our adjusted_gross field is populated where gross is populated
df_movies[(df_movies['adjusted_gross'].isnull()) & (df_movies['gross'].notnull())]

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,release_date,decade,InflationMultiplier,adjusted_gross,adjusted_budget


In [9]:
# ensure our adjusted_budget field is populated where budget is populated
df_movies[(df_movies['adjusted_budget'].isnull()) & (df_movies['budget'].notnull())]

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,release_date,decade,InflationMultiplier,adjusted_gross,adjusted_budget


### Create discrete fields for budget and gross rounded to nearest $1M to reduce dimensionality

In [10]:

# round each gross to the nearest million as integer
df_movies['gross_discrete'] = (df_movies['adjusted_gross'] / 1000000).round() 
df_movies['gross_discrete'].value_counts().sort_index()


gross_discrete
0.0       367
1.0       342
2.0       213
3.0       190
4.0       177
         ... 
2632.0      1
2794.0      1
3497.0      1
4242.0      1
4403.0      1
Name: count, Length: 829, dtype: int64

In [11]:

# do the same for adjusted_budget
df_movies['budget_discrete'] = (df_movies['adjusted_budget'] / 1000000).round() 
df_movies['budget_discrete'].value_counts().sort_index()


budget_discrete
0.0      62
1.0      81
2.0      64
3.0      76
4.0      60
         ..
411.0     1
415.0     1
424.0     1
445.0     1
465.0     1
Name: count, Length: 296, dtype: int64

## Data validity checks

### In order to attribute box office $$ to any particular star, we have to ensure that any one star is recorded exactly the same, e.g. "Robert DeNiro" is not also listed as "Robert De Niro". To do this we will get a list of unique star names and do a vector angle analysis to give similarity score with threshhold >=.75. We need to do the same for writer, director, country, genre and company as well

* the function similarity_check allows us to run this check on individual fields 


In [12]:

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

threshold = 0.80

def similarity_check(field):
    print(f"\nChecking field: {field}")
    unique_values = df_movies[field].dropna().unique()
    tfidf_matrix = TfidfVectorizer().fit_transform(unique_values)
    vectors = tfidf_matrix.toarray()
    cosine_matrix = cosine_similarity(vectors)
    similar_pairs = []
    for i in range(len(unique_values)):
        for j in range(i + 1, len(unique_values)):
            if cosine_matrix[i, j] > threshold and unique_values[i] != unique_values[j]:
                similar_pairs.append((unique_values[i], unique_values[j], cosine_matrix[i, j]))
    for val1, val2, score in similar_pairs:
        print(f"Similar: '{val1}' and '{val2}' with similarity score: {score:.2f}")

# list of fields to loop through to check for similar entries
fields_to_check = ['star', 'writer', 'director', 'country', 'genre', 'company']

for field in fields_to_check:
    similarity_check(field)


Checking field: star
Similar: 'Jason Scott Lee' and 'Jason Lee' with similarity score: 0.82
Similar: 'Michael Jordan' and 'Michael B. Jordan' with similarity score: 1.00

Checking field: writer
Similar: 'Richard Matheson' and 'Richard Christian Matheson' with similarity score: 0.80
Similar: 'Lawrence J. Block' and 'Lawrence Block' with similarity score: 1.00
Similar: 'Bruce Rubin' and 'Bruce Joel Rubin' with similarity score: 0.83
Similar: 'Robert Katz' and 'A L Katz' with similarity score: 0.85
Similar: 'W.S. Gilbert' and 'David Gilbert' with similarity score: 0.86
Similar: 'Daniel Petrie Jr.' and 'Daniel Petrie' with similarity score: 0.84
Similar: 'William Kennedy' and 'William P. Kennedy' with similarity score: 1.00
Similar: 'Peter Prince' and 'Prince' with similarity score: 0.82
Similar: 'Paul Hogan' and 'P.J. Hogan' with similarity score: 0.83
Similar: 'S.S. Wilson' and 'Michael G. Wilson' with similarity score: 0.83
Similar: 'S.S. Wilson' and 'David C. Wilson' with similarity s

### Analysis of the companies reveals that major studios have subsidiaries that should be folded in with their parent companies and/or have evolving names over time which should be historically joined for the purpose of analysis.

* For each of the following updates, I researched if the companies were in fact related and only update if there is a definitive link.

In [13]:
# create new field in df_movies called 'company_grouped' to hold the company name initially
df_movies['company_grouped'] = df_movies['company']

# update company_grouped = 'Huayi Brothers' where company contains 'Huayi Brothers'
df_movies.loc[df_movies['company'].str.contains('Huayi Brothers', na=False), 'company_grouped'] = 'Huayi Brothers'

# update company_grouped = 'Cannon Films' where company contains 'Cannon'
df_movies.loc[df_movies['company'].str.contains('Cannon', na=False), 'company_grouped'] = 'Cannon Films'

# update company_grouped = 'Warner Brothers' where company contains 'Warner Bro'
df_movies.loc[df_movies['company'].str.contains('Warner Bro', na=False), 'company_grouped'] = 'Warner Brothers'

# update company_grouped = 'lucasfilm' where company contains 'lucasfilm'
df_movies.loc[df_movies['company'].str.contains('Lucasfilm', na=False), 'company_grouped'] = 'Luscasfilm'

# update company_grouped = 'United Artists' where company contains 'lucasfilm'
df_movies.loc[df_movies['company'].str.contains('United Artists', na=False), 'company_grouped'] = 'United Artists'

# update company_grouped = 'Walt Disney' where company contains 'Walt Disney'
df_movies.loc[df_movies['company'].str.contains('Walt Disney', na=False), 'company_grouped'] = 'Walt Disney'

# update company_grouped = 'Samuel Goldwyn' where company contains 'Samuel Goldwyn'
df_movies.loc[df_movies['company'].str.contains('Samuel Goldwyn', na=False), 'company_grouped'] = 'Samual Goldwyn'

# update company_grouped = 'Brownstone Productions' where company contains 'brownstone'
df_movies.loc[df_movies['company'].str.contains('Brownstone', na=False), 'company_grouped'] = 'Brownstone Productions'

# update company_grouped = 'HBO' where company contains 'HBO'
df_movies.loc[df_movies['company'].str.contains('HBO', na=False), 'company_grouped'] = 'HBO'

# update company_grouped = 'Polygram' where company contains 'Polygram'
df_movies.loc[df_movies['company'].str.contains('PolyGram', na=False), 'company_grouped'] = 'Polygram Filmed Entertainment'

# update company_grouped = 'Anapurna' where company contains 'Anapurna'
df_movies.loc[df_movies['company'].str.contains('Anapurna', na=False), 'company_grouped'] = 'Anapurna'

# update company_grouped = 'Penta' where company contains 'Anapurna'
df_movies.loc[df_movies['company'].str.contains('Penta', na=False), 'company_grouped'] = 'Penta'

# update company_grouped = 'Ben-Ami/Leeds Productions' where company contains 'Ben-Ami'
df_movies.loc[df_movies['company'].str.contains('Ben-Ami', na=False), 'company_grouped'] = 'Ben-Ami/Leeds Productions'

# update company_grouped = '21st Century Films' where company contains '21st Century Film'
df_movies.loc[df_movies['company'].str.contains('21st Century Films', na=False), 'company_grouped'] = '21st Century Film'

# update company_grouped = 'Lions Gate' where company contains 'Lions Gate'
df_movies.loc[df_movies['company'].str.contains('Lions Gate', na=False), 'company_grouped'] = 'Lions Gate'

# update company_grouped = 'Alliance' where company contains 'Alliance
df_movies.loc[df_movies['company'].str.contains('Alliance', na=False), 'company_grouped'] = 'Alliance'

# update company_grouped = 'Dreamworks' where company contains 'Dreamworks
df_movies.loc[df_movies['company'].str.contains('Dreamworks', na=False), 'company_grouped'] = 'DreamWorks'

# update company_grouped = 'IFC' where company contains 'IFC
df_movies.loc[df_movies['company'].str.contains('IFC', na=False), 'company_grouped'] = 'IFC'

# update company_grouped = 'Abandon' where company contains 'IFC
df_movies.loc[df_movies['company'].str.contains('Abandon', na=False), 'company_grouped'] = 'Abandon'

# update company_grouped = 'Warp' where company contains 'Warp
df_movies.loc[df_movies['company'].str.contains('Warp', na=False), 'company_grouped'] = 'Warp'

# update company_grouped = 'Filmax' where company contains 'Filmax
df_movies.loc[df_movies['company'].str.contains('Filmax', na=False), 'company_grouped'] = 'Filmax'

# update company_grouped = 'Dino De Laurentis' where company contains 'Dino de Laurentiis
df_movies.loc[df_movies['company'].str.contains('Dino De Laurentiis ', na=False), 'company_grouped'] = 'Dino De Laurentis Company'

# update company_grouped = '21st Century' where company contains '21st Century'
df_movies.loc[df_movies['company'].str.contains('21st Century', na=False), 'company_grouped'] = '21st Century'

# update company_grouped = 'BBC' where company contains '21st Century'
df_movies.loc[df_movies['company'].str.contains('BBC', na=False), 'company_grouped'] = 'BBC'

# update company_grouped = 'Embassy' where company contains 'Embassy'
df_movies.loc[df_movies['company'].str.contains('Embassy', na=False), 'company_grouped'] = 'Embassy'

# update company_grouped = 'Guber-Peters Company' where company contains 'Guber-Peters Company'
df_movies.loc[df_movies['company'].str.contains('Guber-Peters Company', na=False), 'company_grouped'] = 'Guber-Peters Company'

# update company_grouped = 'Pathé' where company contains 'Pathé'
df_movies.loc[df_movies['company'].str.contains('Pathé', na=False), 'company_grouped'] = 'Pathé'

# update company_grouped = 'Mirage' where company contains 'Mirage'
df_movies.loc[df_movies['company'].str.contains('Mirage', na=False), 'company_grouped'] = 'Mirage'

# update company_grouped = 'Pressman' where company contains 'Pressman'
df_movies.loc[df_movies['company'].str.contains('Pressman', na=False), 'company_grouped'] = 'Pressman'

# update company_grouped = 'Carolco' where company contains 'Carolco'
df_movies.loc[df_movies['company'].str.contains('Carolco', na=False), 'company_grouped'] = 'Carolco'

# update company_grouped = 'Annapurna' where company contains 'Annapurna'
df_movies.loc[df_movies['company'].str.contains('Annapurna', na=False), 'company_grouped'] = 'Annapurna'

# update company_grouped = 'Constantin' where company contains 'Constantin'
df_movies.loc[df_movies['company'].str.contains('Constantin', na=False), 'company_grouped'] = 'Constantin'

# update company_grouped = 'New World' where company contains 'New World'
df_movies.loc[df_movies['company'].str.contains('New World', na=False), 'company_grouped'] = 'New World'

# update company_grouped = 'Fidélité' where company contains 'Fidélité'
df_movies.loc[df_movies['company'].str.contains('Fidélité', na=False), 'company_grouped'] = 'Fidélité'

# update company_grouped = 'Mandalay' where company contains 'Mandalay'
df_movies.loc[df_movies['company'].str.contains('Mandalay', na=False), 'company_grouped'] = 'Mandalay'

# update company_grouped = 'Twentieth Century Fox' where company contains 'Twentieth Century Fox'
df_movies.loc[df_movies['company'].str.contains('Twentieth Century Fox', na=False), 'company_grouped'] = 'Twentieth Century Fox'

# update company_grouped = 'New Century' where company contains 'New Century'
df_movies.loc[df_movies['company'].str.contains('New Century', na=False), 'company_grouped'] = 'New Century'

# update company_grouped = 'In-Gear' where company contains 'In-Gear'
df_movies.loc[df_movies['company'].str.contains('In-Gear', na=False), 'company_grouped'] = 'In-Gear'

# update company_grouped = 'STX' where company contains 'STX'
df_movies.loc[df_movies['company'].str.contains('STX', na=False), 'company_grouped'] = 'STX'

# update company_grouped = 'Channel Four' where company contains 'Channel Four'
df_movies.loc[df_movies['company'].str.contains('Channel Four', na=False), 'company_grouped'] = 'Channel Four'

# update company_grouped = 'Columbia' where company contains 'Columbia'
df_movies.loc[df_movies['company'].str.contains('Columbia Films', na=False), 'company_grouped'] = 'Columbia Pictures'

# update company_grouped = 'Gaumont' where company contains 'Gaumont'
df_movies.loc[df_movies['company'].str.contains('Gaumont', na=False), 'company_grouped'] = 'Gaumont'

# update company_grouped = 'NBC' where company contains 'NBC'
df_movies.loc[df_movies['company'].str.contains('NBC Film', na=False), 'company_grouped'] = 'NBC Productions'

# update company_grouped = 'Panorama' where company contains 'Panorama'
df_movies.loc[df_movies['company'].str.contains('Panorama', na=False), 'company_grouped'] = 'Panorama'

# update company_grouped = 'BET' where company contains 'BET'
df_movies.loc[df_movies['company'].str.contains('BET Pictures', na=False), 'company_grouped'] = 'BET Films'

# update company_grouped = 'Imagine Entertainment' where company contains 'Imagine Films'
df_movies.loc[df_movies['company'].str.contains('Imagine Films', na=False), 'company_grouped'] = 'Imagine Entertainment'

# update company_grouped = 'New Visions' where company contains 'New Visions'
df_movies.loc[df_movies['company'].str.contains('New Visions', na=False), 'company_grouped'] = 'New Visions'

# update company_grouped = 'ERBP' where company contains 'erbp'
df_movies.loc[df_movies['company'].str.contains('erbp', na=False), 'company_grouped'] = 'ERBP'

# similarity_check('company_grouped')

## Scope Considerations

### Determine how many movies are foreign vs. domestic

In [14]:
# give me a count of records from df_movies where country is or is not 'United States'
df_movies['is_domestic'] = df_movies['country'] == 'United States'
df_movies['is_domestic'].value_counts()


is_domestic
True     6735
False     933
Name: count, dtype: int64

#### there are 6735 domestic movies and 933 foreign movies, meaning we have a critical mass of data even without the foreign movies.  The decision was made to remove them.

In [15]:
#delete from df_movies where country <> 'United States'
df_movies = df_movies[df_movies['country'] == 'United States']
#give me a count of rows by country
df_movies['country'].value_counts()

country
United States    6735
Name: count, dtype: int64

### Movies with null budget or gross


In [16]:
# give me a count of rows with null values in budget or gross as separate numbers with formatted output of "Null Budget: X, Null Gross: Y"
null_budget_count = df_movies['budget'].isnull().sum()
null_gross_count = df_movies['gross'].isnull().sum()
print(f"Null Budget: {null_budget_count}, Null Gross: {null_gross_count}")
                                                                                                                                   

Null Budget: 1607, Null Gross: 132



#### only 132 movies have null gross values, and we cannot consider them for analysis
#### 1607 movies have null budgets, but we will leave them in case budget does not have a significant impact on gross

In [17]:
#delete from df_movies where gross is null
df_movies = df_movies[~df_movies['gross'].isnull()]
df_movies.shape

(6603, 24)

## write cleaned dataset to 

In [18]:
df_movies.to_csv('./data/processed/movies_cleaned.csv', index=False)
df_movies.to_csv('./data/processed/movies_model.csv', index=False)